In [42]:
! git clone --depth 1 https://github.com/rwightman/pytorch-image-models.git
%cd pytorch-image-models/results

In [43]:
import pandas as pd

In [67]:
df_result = pd.read_csv('./results-imagenet.csv')
df_result['model'] = df_result['model'].str.split('.').str[0]
df_result

,model,top1,top1_err,top5,top5_err,param_count,img_size,crop_pct,interpolation
0,eva02_large_patch14_448,90.052,9.948,99.048,0.952,305.08,448,1.000,bicubic
1,eva02_large_patch14_448,89.970,10.030,99.012,0.988,305.08,448,1.000,bicubic
2,eva_giant_patch14_560,89.786,10.214,98.992,1.008,"1,014.45",560,1.000,bicubic
3,eva02_large_patch14_448,89.622,10.378,98.950,1.050,305.08,448,1.000,bicubic
4,eva02_large_patch14_448,89.574,10.426,98.924,1.076,305.08,448,1.000,bicubic
...,...,...,...,...,...,...,...,...,...
1075,efficientvit_m0,63.270,36.730,85.176,14.824,2.35,224,0.875,bicubic
1076,lcnet_050,63.138,36.862,84.382,15.618,1.88,224,0.875,bicubic
1077,tf_mobilenetv3_small_minimal_100,62.894,37.106,84.238,15.762,2.04,224,0.875,bilinear
1078,tinynet_e,59.866,40.134,81.762,18.238,2.04,106,0.875,bicubic


In [45]:
df = pd.read_csv('./benchmark-infer-amp-nhwc-pt111-cu113-rtx3090.csv')
df

,model,infer_samples_per_sec,infer_step_time,infer_batch_size,infer_img_size,param_count
0,tinynet_e,68298.73,14.982,1024,106,2.04
1,mobilenetv3_small_050,48773.32,20.985,1024,224,1.59
2,lcnet_035,47045.94,21.755,1024,224,1.64
3,lcnet_050,41541.83,24.639,1024,224,1.88
4,mobilenetv3_small_075,37803.23,27.076,1024,224,2.04
...,...,...,...,...,...,...
737,nfnet_f7,20.63,9307.125,192,608,499.50
738,resnetv2_152x4_bitm,18.31,3496.111,64,480,936.53
739,swin_v2_cr_giant_384,13.76,2325.078,32,384,2598.76
740,cait_m48_448,13.51,9473.095,128,448,356.46


In [71]:
def get_data(part, col):
    df = pd.read_csv(f'benchmark-{part}-amp-nchw-pt111-cu113-rtx3090.csv').merge(df_result, on='model')
    df['secs'] = 1. / df[col]
    df['family'] = df.model.str.extract('^([a-z]+?(?:v2)?)(?:\d|_|$)')
    df = df[~df.model.str.endswith('gn')]
    df.loc[df.model.str.contains('in22'),'family'] = df.loc[df.model.str.contains('in22'),'family'] + '_in22'
    df.loc[df.model.str.contains('resnet.*d'),'family'] = df.loc[df.model.str.contains('resnet.*d'),'family'] + 'd'
    return df[df.family.str.contains('^re[sg]netd?|beit|convnext|levit|efficient|vit|vgg')]

In [72]:
df = get_data('infer', 'infer_samples_per_sec')

In [73]:
df

,model,infer_samples_per_sec,infer_step_time,infer_batch_size,infer_img_size,param_count_x,top1,top1_err,top5,top5_err,param_count_y,img_size,crop_pct,interpolation,secs,family
10,levit_128s,19963.52,51.282,1024,224,7.78,76.526,23.474,92.872,7.128,7.78,224,0.900,bicubic,0.000050,levit
13,regnetx_002,16238.56,63.048,1024,224,2.68,68.752,31.248,88.542,11.458,2.68,224,0.875,bicubic,0.000062,regnetx
14,regnety_002,15227.23,67.235,1024,224,3.16,70.280,29.720,89.530,10.470,3.16,224,0.875,bicubic,0.000066,regnety
17,levit_128,13679.77,74.845,1024,224,9.21,78.490,21.510,94.012,5.988,9.21,224,0.900,bicubic,0.000073,levit
18,vit_small_patch32_224,12109.88,84.548,1024,224,22.88,75.994,24.006,93.270,6.730,22.88,224,0.900,bicubic,0.000083,vit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,resnetrs350,151.88,3371.146,512,384,163.96,84.714,15.286,96.992,3.008,163.96,384,1.000,bicubic,0.006584,resnetrs
640,resnetrs420,108.78,4706.738,512,416,191.89,85.004,14.996,97.124,2.876,191.89,416,1.000,bicubic,0.009193,resnetrs
643,vit_large_patch16_384,94.74,2702.220,256,384,304.72,87.084,12.916,98.302,1.698,304.72,384,1.000,bicubic,0.010555,vit
644,beit_large_patch16_384,82.64,3097.749,256,384,305.00,88.402,11.598,98.608,1.392,305.00,384,1.000,bicubic,0.012101,beit


In [74]:
import plotly.express as px

w, h = 1000, 800

def show_all(df, title, size):
    return px.scatter(df, width=w, height=h, size=df[size]**2, title=title,
                      x='secs',  y='top1', log_x=True,
                      color='family', hover_name='model', hover_data=[size])


show_all(df, 'Inference', 'infer_img_size')

In [75]:
subs = 'levit|resnetd?|regnetx|vgg|convnext.*|efficientnetv2|beit'
def show_subs(df, title, size):
    df_subs = df[df.family.str.fullmatch(subs)]
    return px.scatter(df_subs, width=w, height=h, size=df_subs[size]**2, title=title,
        trendline="ols", trendline_options={'log_x':True},
        x='secs',  y='top1', log_x=True, color='family', hover_name='model', hover_data=[size])

In [76]:
show_subs(df, 'Inference', 'infer_img_size')